In [30]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy

In [31]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

### Halloween Challenge
Find the best solution with the fewest calls to the fitness functions for:

- num_points = [100, 1_000, 5_000]
- num_sets = num_points
- density = [.3, .7]

In [32]:
N_POINTS = 1000
N_SETS = N_POINTS
DENSITY = .3
x = make_set_covering_problem(N_POINTS, N_SETS, DENSITY)
SETS = x.toarray()
# print(SETS)
# print("Element at row=42 and column=42:", x[42, 42])

In [33]:
def goal_check(state):
    progress = [False for _ in range(N_POINTS)]
    for i, set in enumerate(state):
        if set:
            list_set = SETS[i].tolist()
            for i in range(N_POINTS):
                progress[i] = progress[i] or list_set[i]
    
    for element in progress:
        if not element:
            return False
    return True

In [34]:
# returns true if the set at the given index cover at least a percent of elements equal to DENSITY
def is_good(index):
    return sum(SETS[index]) >= DENSITY * N_POINTS


# returns true if the set at the given index is worst than the top {treshold}% of taken sets
def could_be_better(state, index, treshold=0.5):
    better_count = 0
    for i, set in enumerate(state):
        if set and sum(SETS[i]) > sum(SETS[index]):
            better_count += 1
    if (better_count / N_POINTS) >= treshold:
        return True
    else:
        return False
        

# removes redundant sets, starting with the ones with less covering
def prune(state):
    sets_used = []
    for i, taken in enumerate(state):
        if taken:
            sets_used.append([SETS[i], i, sum(SETS[i])])
    sets_used.sort(key=lambda x: x[1])

    # check if each set is redundant (stop at the first found)
    for taken_set in sets_used:
        redundant = True
        for i, val in enumerate(taken_set[0]):
            if val:
                redundancy = 0
                for set in sets_used:
                    redundancy += set[0][i]
                if redundancy < 2:
                    redundant = False
                    break
        if redundant:
            return set[1]
    return False


In [35]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(N_POINTS)]),
        )
    )
    return valid, -cost

In [36]:
# the default tweak function, randomly chooses one element to add or remove
def random_tweak(state):
    new_state = copy(state)
    index = randint(0, N_POINTS - 1)
    new_state[index] = not new_state[index]
    return new_state


# always add a new set, then prunes the state removing redundant sets
def tweak_1(state):
    new_state = copy(state)
    
    # loops until it picks a non taken set (choosing randomly)
    while 1:
        index = randint(0, N_POINTS - 1)
        if not new_state[index]:
            break
    new_state[index] = True

    if(not goal_check(new_state)):
        return new_state

    # eliminate redundant sets until there are no more
    while 1:
        to_prune = prune(new_state)
        if not to_prune:
            return new_state
        new_state[to_prune] = False


In [37]:
print('Default random tweak function, used for benchmarking')
current_state = [False for _ in range(N_POINTS)]
print(f'fitness: {fitness(current_state)[0]}\t sets taken: {-fitness(current_state)[1]}\t steps: 0')

fitness_calls = 0
updates = 0

for step in range(10_000):
    new_state = random_tweak(current_state)
    fitness_calls += 1
    if fitness(new_state) >= fitness(current_state):
        updates += 1
        current_state = new_state
        print(f'fitness: {fitness(current_state)[0]}\t sets taken: {-fitness(current_state)[1]}\t steps: {step + 1}')

print()
print(f'Fitness calls: {fitness_calls}')
print(f'Updates: {updates}')

Default random tweak function, used for benchmarking
fitness: 0	 sets taken: 0	 steps: 0
fitness: 321	 sets taken: 1	 steps: 1
fitness: 511	 sets taken: 2	 steps: 2
fitness: 658	 sets taken: 3	 steps: 3
fitness: 755	 sets taken: 4	 steps: 4
fitness: 808	 sets taken: 5	 steps: 5
fitness: 874	 sets taken: 6	 steps: 6
fitness: 909	 sets taken: 7	 steps: 7
fitness: 936	 sets taken: 8	 steps: 8
fitness: 955	 sets taken: 9	 steps: 9
fitness: 974	 sets taken: 10	 steps: 10
fitness: 986	 sets taken: 11	 steps: 11
fitness: 987	 sets taken: 12	 steps: 12
fitness: 990	 sets taken: 13	 steps: 13
fitness: 993	 sets taken: 14	 steps: 14
fitness: 995	 sets taken: 15	 steps: 15
fitness: 996	 sets taken: 16	 steps: 17
fitness: 998	 sets taken: 17	 steps: 18
fitness: 999	 sets taken: 18	 steps: 20
fitness: 1000	 sets taken: 19	 steps: 22
fitness: 1000	 sets taken: 18	 steps: 92
fitness: 1000	 sets taken: 17	 steps: 96
fitness: 1000	 sets taken: 16	 steps: 97
fitness: 1000	 sets taken: 15	 steps: 823

Fi

In [39]:
print('My function')
current_state = [False for _ in range(N_POINTS)]
print(f'fitness: {fitness(current_state)[0]}\t sets taken: {-fitness(current_state)[1]}\t steps: 0')

fitness_calls = 0
updates = 0

for step in range(10_000):
    #if goal_check(current_state):
    #    break
    new_state = tweak_1(current_state)
    fitness_calls += 1
    if fitness(new_state) > fitness(current_state):
        updates += 1
        current_state = new_state
        print(f'fitness: {fitness(current_state)[0]}\t sets taken: {-fitness(current_state)[1]}\t steps: {step + 1}')

print()
print(f'Fitness calls: {fitness_calls}')
print(f'Updates: {updates}')

My function
fitness: 0	 sets taken: 0	 steps: 0
fitness: 286	 sets taken: 1	 steps: 1
fitness: 491	 sets taken: 2	 steps: 2
fitness: 630	 sets taken: 3	 steps: 3
fitness: 742	 sets taken: 4	 steps: 4
fitness: 816	 sets taken: 5	 steps: 5
fitness: 859	 sets taken: 6	 steps: 6
fitness: 894	 sets taken: 7	 steps: 7
fitness: 931	 sets taken: 8	 steps: 8
fitness: 951	 sets taken: 9	 steps: 9
fitness: 967	 sets taken: 10	 steps: 10
fitness: 980	 sets taken: 11	 steps: 11
fitness: 989	 sets taken: 12	 steps: 12
fitness: 991	 sets taken: 13	 steps: 13
fitness: 994	 sets taken: 14	 steps: 14
fitness: 997	 sets taken: 15	 steps: 15
fitness: 998	 sets taken: 16	 steps: 18
fitness: 999	 sets taken: 17	 steps: 24


KeyboardInterrupt: 